# Data Preparation

In [80]:
import pandas
import matplotlib.pyplot as plt
from collections import Counter
import string
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import losses

wf = pandas.read_csv("/home/yash/Code/PresidentialSpeechClassification/EMPOLITICON.csv")
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,16/07/2021,Vladimir Putin,Meeting of APEC Economic Leaders,"Madam Chair,\n\nColleagues,\n\nFirst of all, ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,OPTIMISM,DEVELOPMENT
1,Russia,2021-09-05 00:00:00,Vladimir Putin,Victory Parade on Red Square,"Citizens of Russia,\n\nDear veterans,\n\nComra...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,JOY,NATIONALISM
2,Russia,2021-08-04 00:00:00,Vladimir Putin,Meeting on the results of implementing Preside...,"Good afternoon, colleagues.\n\nLet’s start.\n\...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
3,Russia,21-11-2020,Vladimir Putin,G20 Summit,"Colleagues,\n\nThe scope of problems humanity ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
4,Russia,20-11-2020,Vladimir Putin,Address to participants in Nuremberg Lessons f...,"Colleagues, friends,\n\nFirst of all, I would ...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,UPSET,EXTREMISM
...,...,...,...,...,...,...,...,...,...,...
2005,USA,05/02/2013,Barack Obama,\nRemarks by the President,"Good afternoon, everybody. \n\nI wanted to sa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,DEVELOPMENT
2006,USA,04/02/2013,Barack Obama,\nRemarks by the President on Preventing Gun V...,"Hello, everybody. Please have a seat. Have a...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,OTHERS
2007,USA,02/02/2013,Barack Obama,\nWeekly Address: A Balanced Approach to Growi...,"Hi, everybody. \n\nIn the coming weeks, we fa...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,OPTIMISM,DEVELOPMENT
2008,USA,28/01/2013,Barack Obama,\nRemarks by the President Before Meeting with...,"Well, Vice President Biden and I just want to...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,OPTIMISM,OTHERS


In [81]:
# shuffing the data
wf = wf.sample(frac=1).reset_index(drop=True)
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,2021-03-18 00:00:00,Joe Biden,Remarks by President Biden on the 100 Million ...,"THE PRESIDENT: Well, folks, good to see you a...",President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,OPTIMISM,DEVELOPMENT
1,Russia,14-11-2009,Dmitry Medvedev,Speech at APEC CEO Summit,"Ladies and gentlemen,\n\nI would like first of...",President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT
2,USA,31/08/2013,Barack Obama,\nWeekly Address: Commemorating Labor Day,"Hi, everybody. This Labor Day weekend, as we...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,JOY,OTHERS
3,USA,2017-11-23 00:00:00,Donald Trump,Remarks by President Trump to Members of the M...,"THE PRESIDENT: Hello, everybody, and happy Th...",President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,JOY,NATIONALISM
4,USA,11/05/2013,Barack Obama,\nWeekly Address: Growing the Housing Market a...,"Hi, everybody. Our top priority as a nation ...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,JOY,DEVELOPMENT
...,...,...,...,...,...,...,...,...,...,...
2005,USA,07/03/2015,Barack Obama,\nWeekly Address: Let Girls Learn,"Hi, everybody. Sunday is International Women’...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,OTHERS
2006,USA,2021-06-07 00:00:00,Joe Biden,"Statement by President Joseph R. Biden, Jr. on...","Today, the Department of Justice will file a b...",President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,NEUTRAL,OTHERS
2007,Russia,2001-12-06 00:00:00,Vladimir Putin,Address at a Meeting with NGO Representatives,"Good afternoon!\n\nTo begin with, I want to c...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,JOY,OTHERS
2008,Russia,21-02-2001,Vladimir Putin,Excerpt from a Speech at a Meeting of the Stat...,The land issue has historically been a thorny...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,NEUTRAL,DEVELOPMENT


In [82]:
def count_unique_word(wf):
    count = Counter()
    for word in wf['Emotion']:
        count[word] += 1
    return count

counter = count_unique_word(wf)

In [83]:
print(counter.most_common())

[('NEUTRAL', 589), ('OPTIMISM', 583), ('JOY', 525), ('UPSET', 313)]


Setting labels for a multi-classification problem

In [84]:
#labeling the emotion column such that the most common emotion is 0, the second most common is 1 and so on
for i in range(len(wf)):
    if wf['Emotion'][i] == counter.most_common()[0][0]:
        wf['Emotion'][i] = 0
    elif wf['Emotion'][i] == counter.most_common()[1][0]:
        wf['Emotion'][i] = 1
    elif wf['Emotion'][i] == counter.most_common()[2][0]:
        wf['Emotion'][i] = 2
    elif wf['Emotion'][i] == counter.most_common()[3][0]:
        wf['Emotion'][i] = 3

#these labels will be used to train the model

/tmp/ipykernel_113036/1047653752.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Emotion'][i] = 1
/tmp/ipykernel_113036/1047653752.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting valu

In [85]:
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,2021-03-18 00:00:00,Joe Biden,Remarks by President Biden on the 100 Million ...,"THE PRESIDENT: Well, folks, good to see you a...",President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,1,DEVELOPMENT
1,Russia,14-11-2009,Dmitry Medvedev,Speech at APEC CEO Summit,"Ladies and gentlemen,\n\nI would like first of...",President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
2,USA,31/08/2013,Barack Obama,\nWeekly Address: Commemorating Labor Day,"Hi, everybody. This Labor Day weekend, as we...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,OTHERS
3,USA,2017-11-23 00:00:00,Donald Trump,Remarks by President Trump to Members of the M...,"THE PRESIDENT: Hello, everybody, and happy Th...",President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,2,NATIONALISM
4,USA,11/05/2013,Barack Obama,\nWeekly Address: Growing the Housing Market a...,"Hi, everybody. Our top priority as a nation ...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,DEVELOPMENT
...,...,...,...,...,...,...,...,...,...,...
2005,USA,07/03/2015,Barack Obama,\nWeekly Address: Let Girls Learn,"Hi, everybody. Sunday is International Women’...",President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,OTHERS
2006,USA,2021-06-07 00:00:00,Joe Biden,"Statement by President Joseph R. Biden, Jr. on...","Today, the Department of Justice will file a b...",President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,0,OTHERS
2007,Russia,2001-12-06 00:00:00,Vladimir Putin,Address at a Meeting with NGO Representatives,"Good afternoon!\n\nTo begin with, I want to c...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,2,OTHERS
2008,Russia,21-02-2001,Vladimir Putin,Excerpt from a Speech at a Meeting of the Stat...,The land issue has historically been a thorny...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT


In [86]:
# wf = wf.head(739)

In [87]:
wf.iloc[76]['Text_of_Speech']

'Prime Minister Ten weeks ago we pressed pause on our NHS reforms. We wanted to speak to professionals, patients and everyone who cares about the NHS… …and make sure our proposals were absolutely right. Now there were those who said this was a humiliating U-turn, that we were back-tracking and ditching all our plans. And there were those who said the opposite… …that actually we weren’t going to change anything - that this was all a big PR stunt. Today we show that both are wrong. The fundamentals of our plans… …more control to patients, more power to doctors and nurses, less bureaucracy in the NHS… …they are as strong today as they’ve ever been. But the shape of our plans… …the detail of how we’re going to make all this work… …that really has changed - as a direct result of this consultation. We listened to what professionals and patients told us. You wanted us to make clear that competition isn’t there for its own sake, but to make life better for patients - done. You wanted us to get

In [88]:
## one thing we can do is to use the model from the presidential classifier and then, if a given speech is classified
## as angry or some other negative mood, and it includes the word "US, NATO", or something related to the west, we classify it as adversarial

In [89]:
#removing the punctuation

## Yet to apply this to the data

def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

In [90]:
#removing the stop words (commonly used words that do not add much meaning to a sentence)

## Yet to actually apply this function to the data

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    #removes stop words
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)


[nltk_data] Downloading package stopwords to /home/yash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [91]:
#cleaning up the speeches so that common stopwords and puntuation are removed
for i in range(len(wf['Text_of_Speech'])):
    wf['Text_of_Speech'][i] = remove_punctuation(wf['Text_of_Speech'][i])
    wf['Text_of_Speech'][i] = remove_stopwords(wf['Text_of_Speech'][i])

/tmp/ipykernel_113036/2134825500.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Text_of_Speech'][i] = remove_punctuation(wf['Text_of_Speech'][i])
/tmp/ipykernel_113036/2134825500.py:4: FutureWarning: ChainedAssignmentError: behaviou

In [92]:
#drop rows without string in speech
for i in range(len(wf) - 1, -1, -1):
        if not (isinstance(wf.iloc[i]['Text_of_Speech'], str)):
                wf = wf.drop(i)

In [93]:
#counting the number of unique words in each speech


def count_unique_word(wf_training):
    count = Counter()
    for i in range(len(wf_training)):
        for word in wf_training.iloc[i]['Text_of_Speech'].split():
            count[word] += 1
    return count

counter = count_unique_word(wf)

In [94]:
counter

Counter({'people': 9977,
         '—': 8222,
         'new': 6587,
         'world': 6530,
         'work': 6325,
         'us': 6261,
         'china': 6246,
         'countries': 6156,
         'development': 6030,
         'one': 5367,
         'also': 5297,
         'country': 5190,
         'cooperation': 5139,
         'today': 4997,
         'need': 4856,
         'make': 4751,
         'thank': 4400,
         'years': 4319,
         'like': 4236,
         'must': 4210,
         'going': 4136,
         'want': 4067,
         'know': 4061,
         'would': 3998,
         'it’s': 3995,
         'time': 3944,
         'international': 3876,
         'economic': 3761,
         'many': 3709,
         'applause': 3698,
         'first': 3525,
         'we’re': 3520,
         '–': 3500,
         'year': 3467,
         'security': 3461,
         'united': 3394,
         'economy': 3278,
         'states': 3250,
         'that’s': 3248,
         'every': 3222,
         'great': 3150,
  

In [95]:
num_unique_words = len(counter)

In [96]:
print(len(wf))

2010


In [97]:
wf_training = wf[:int(len(wf) * 0.8)]
wf_testing = wf[int(len(wf) * 0.8):]

In [98]:
wf_testing.reset_index(inplace=True)
wf_testing.drop('index', axis=1, inplace=True)
wf_testing

/tmp/ipykernel_113036/3894747268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_testing.drop('index', axis=1, inplace=True)


,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,Russia,24-08-2009,Dmitry Medvedev,Concluding Remarks at Meeting on the Siberian ...,would like conclude remarks certain issues yet...,President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,0,OTHERS
1,China,2013-07-20 00:00:00,Xi Jinping,Message of Congratulations From President Xi J...,occasion opening ceremony eco forum annual glo...,President,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,1,DEVELOPMENT
2,Russia,13-09-2004,Vladimir Putin,Speech at the Enlarged Government Meeting with...,dear colleagues meeting unusual format due spe...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,3,EXTREMISM
3,USA,20/11/2014,Barack Obama,\nRemarks by the President in Address to the N...,fellow americans tonight i’d like talk immigra...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,OTHERS
4,USA,03/12/2013,Barack Obama,\nRemarks by the President on the Affordable C...,thanks monica thanks everybody standing behind...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,DEVELOPMENT
...,...,...,...,...,...,...,...,...,...,...
397,USA,07/03/2015,Barack Obama,\nWeekly Address: Let Girls Learn,hi everybody sunday international women’s day ...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,OTHERS
398,USA,2021-06-07 00:00:00,Joe Biden,"Statement by President Joseph R. Biden, Jr. on...",today department justice file brief supreme co...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,0,OTHERS
399,Russia,2001-12-06 00:00:00,Vladimir Putin,Address at a Meeting with NGO Representatives,good afternoon begin want congratulate day rus...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,2,OTHERS
400,Russia,21-02-2001,Vladimir Putin,Excerpt from a Speech at a Meeting of the Stat...,land issue historically thorny issue russia al...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT


In [99]:
wf_training.reset_index(inplace=True)
wf_training.drop('index', axis=1, inplace=True)
wf_training

/tmp/ipykernel_113036/1694703126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_training.drop('index', axis=1, inplace=True)


,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,2021-03-18 00:00:00,Joe Biden,Remarks by President Biden on the 100 Million ...,president well folks good see announced early ...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/speec...,1,DEVELOPMENT
1,Russia,14-11-2009,Dmitry Medvedev,Speech at APEC CEO Summit,ladies gentlemen would like first thank opport...,President,Dmitry Medvedev,http://en.kremlin.ru/events/president/transcri...,0,DEVELOPMENT
2,USA,31/08/2013,Barack Obama,\nWeekly Address: Commemorating Labor Day,hi everybody labor day weekend gather family f...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,OTHERS
3,USA,2017-11-23 00:00:00,Donald Trump,Remarks by President Trump to Members of the M...,president hello everybody happy thanksgiving y...,President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,2,NATIONALISM
4,USA,11/05/2013,Barack Obama,\nWeekly Address: Growing the Housing Market a...,hi everybody top priority nation reigniting tr...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,2,DEVELOPMENT
...,...,...,...,...,...,...,...,...,...,...
1603,USA,2021-12-01 00:00:00,Joe Biden,A Message to the Congress on the Agreement bet...,pleased transmit congress pursuant section 123...,President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,1,INTERNATIONAL AFFAIRS
1604,Russia,2019-03-12 00:00:00,Vladimir Putin,Meeting with senior Defence Ministry officials...,good afternoon colleagues continuing series me...,President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,0,INTERNATIONAL AFFAIRS
1605,USA,09/02/2013,Barack Obama,\nWeekly Address: Averting the Sequester and F...,hi everybody last years democrats republicans ...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,0,DEVELOPMENT
1606,USA,10/09/2013,Barack Obama,\nRemarks by the President in Address to the N...,fellow americans tonight want talk syria matte...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,3,INTERNATIONAL AFFAIRS


In [100]:
training_sentences = wf_training.Text_of_Speech.to_numpy()
#training labels
training_labels = wf_training.Emotion.to_numpy()

validation_sentences = wf_testing.Text_of_Speech.to_numpy()
#training labels
validation_labels = wf_testing.Emotion.to_numpy()

In [101]:
training_sentences.shape, validation_sentences.shape

((1608,), (402,))

In [102]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(training_sentences)

In [103]:
word_index = tokenizer.word_index

In [104]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

In [105]:
#padding sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

#max number of words per sequence
max_length = 1000

training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating="post")
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding="post", truncating="post")
training_padded.shape, validation_padded.shape

((1608, 1000), (402, 1000))

In [106]:
#LSTM Model
import tensorflow.keras as keras


model = keras.models.Sequential()

#input layer
#embedding layer converts the words into vectors of fixed size
model.add(layers.Embedding(num_unique_words, 32))
#dropout rate drops out a certain percentage of input data so that the
#model learns more robust ways of learning and does not overfit
model.add(layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [107]:
loss = 'sparse_categorical_crossentropy'
optim = keras.optimizers.Adam(learning_rate=0.001) #from 0.001
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [108]:
print(training_padded.dtype, training_padded.shape)

int32 (1608, 1000)


In [109]:

training_labels = np.array(training_labels).astype(np.int64)
validation_labels = np.array(validation_labels).astype(np.int64)

In [110]:
print(training_padded.dtype, training_padded.shape)
print(training_labels.dtype, training_labels.shape)

int32 (1608, 1000)
int64 (1608,)


In [111]:
print(training_padded)

[[  59   56  292 ...    0    0    0]
 [ 534  513   24 ...    0    0    0]
 [1847  177 1318 ...    0    0    0]
 ...
 [1847  177   68 ...    0    0    0]
 [ 764   77 1366 ... 4571  405  383]
 [  46  686  437 ... 1897   82  100]]


In [112]:
##making sure that overrepresentation of a class does not affect the model
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(training_labels),
    y=training_labels
)
class_weights = dict(enumerate(class_weights))

In [113]:
model.fit(training_padded, training_labels, epochs=20, validation_data=(validation_padded, validation_labels), class_weight=class_weights, verbose=2)

Epoch 1/20


KeyboardInterrupt: 

# Changing to a Binary Classification Model

In [ ]:
wf = pandas.read_csv("/home/yash/Code/PresidentialSpeechClassification/EMPOLITICON.csv")
wf = wf.sample(frac=1).reset_index(drop=True)
wf

,Country,Date,Speaker,Headline,Text_of_Speech,Designation,Running President/PM,Speech Link,Emotion,Context
0,USA,2017-12-18 00:00:00,Donald Trump,Remarks by President Trump on the Administrati...,THE PRESIDENT: Thank you very much. Thank yo...,President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,OPTIMISM,INTERNATIONAL AFFAIRS
1,Russia,2020-01-03 00:00:00,Vladimir Putin,Meeting on most pressing international issues,"Good afternoon, colleagues,\n\nWe know well th...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,UPSET,DEVELOPMENT
2,USA,2017-03-18 00:00:00,Donald Trump,President Donald J. Trump’s Weekly Address,"My fellow Americans,\n\nThis week, I traveled ...",President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,OPTIMISM,DEVELOPMENT
3,United Kingdom,11 February 2011,David Cameron,PM's Speech on Government Procurement,"Prime Minister Today, we are announcing big ch...",Prime Minister,David Cameron,https://www.gov.uk/government/speeches/pms-spe...,OPTIMISM,DEVELOPMENT
4,Russia,27-12-2007,Vladimir Putin,Opening Address at the Concluding Government M...,"Good afternoon, dear colleagues!\n\nPrime Min...",President,Vladimir Putin,http://en.kremlin.ru/events/president/transcri...,JOY,DEVELOPMENT
...,...,...,...,...,...,...,...,...,...,...
2005,USA,2021-06-26 00:00:00,Joe Biden,"Statement by President Joseph R. Biden, Jr. on...","Today, on the anniversary of the United Nation...",President,Joe Biden,https://www.whitehouse.gov/briefing-room/state...,NEUTRAL,OTHERS
2006,China,2020-07-20 00:00:00,H.E. Wang Yi,Study and Implement Xi Jinping Thought on Dipl...,"Colleagues and guests,\n\nToday is an importan...",State Councilor and Foreign Minister of the Pe...,Xi Jinping,https://www.fmprc.gov.cn/mfa_eng/wjdt_665385/z...,NEUTRAL,OTHERS
2007,USA,2018-01-10 00:00:00,Donald Trump,Remarks by President Trump in Cabinet Meeting,THE PRESIDENT: Welcome back to the studio. N...,President,Donald Trump,https://trumpwhitehouse.archives.gov/briefings...,OPTIMISM,DEVELOPMENT
2008,USA,27/05/2013,Barack Obama,\nRemarks by the President Commemorating Memor...,Thank you. (Applause.) Please be seated. Th...,President,Barack Obama,https://obamawhitehouse.archives.gov/the-press...,NEUTRAL,OTHERS


In [ ]:
def count_unique_word(wf):
    count = Counter()
    for word in wf['Emotion']:
        count[word] += 1
    return count

counter = count_unique_word(wf)

In [ ]:
print(counter.most_common())

[('NEUTRAL', 589), ('OPTIMISM', 583), ('JOY', 525), ('UPSET', 313)]


In [ ]:
#made a change so that NEUTRAL and UPSET are 0, OPTIMISM and JOY are 1
for i in range(len(wf)):
    if wf['Emotion'][i] == counter.most_common()[0][0]:
        wf['Emotion'][i] = 0
    elif wf['Emotion'][i] == counter.most_common()[1][0]:
        wf['Emotion'][i] = 1
    elif wf['Emotion'][i] == counter.most_common()[2][0]:
        wf['Emotion'][i] = 1
    elif wf['Emotion'][i] == counter.most_common()[3][0]:
        wf['Emotion'][i] = 0

/tmp/ipykernel_113036/2840128132.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  wf['Emotion'][i] = 1
/tmp/ipykernel_113036/2840128132.py:10: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting val